In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Atherosclerosis/GSE123088'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True  # Assuming that gene expression data is available in this dataset
trait_row = 1 if len(set(['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'])) > 1 else None
age_row = 3 if len(set(['age: 56', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 'age: 80', 'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 'age: 81', 'age: 70', 'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 'age: 45', 'age: 65', 'age: 48', 'age: 50', 'age: 24', 'age: 42'])) > 1 else None
gender_row = 2 if len(set(['Sex: Male', 'Sex: Female'])) > 1 else None

def convert_trait(value):
    val = value.split(': ')[1].strip().upper() if ': ' in value else value.strip().upper()
    return 1 if val == 'ATHEROSCLEROSIS' else 0 if val in ['HEALTHY_CONTROL', 'CONTROL'] else None

def convert_age(value):
    val = value.split(': ')[1].strip() if ': ' in value else value.strip()
    return float(val) if val.isdigit() else None

def convert_gender(value):
    val = value.split(': ')[1].strip().upper() if ': ' in value else value.strip().upper()
    return 1 if val == 'MALE' else 0 if val == 'FEMALE' else None

save_cohort_info('GSE123088', './preprocessed/Atherosclerosis/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Atherosclerosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Atherosclerosis/trait_data/GSE123088.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3494884': [nan, 56.0, 1.0], 'GSM3494885': [None, None, None], 'GSM3494886': [nan, 20.0, 0.0], 'GSM3494887': [nan, 51.0, 0.0], 'GSM3494888': [nan, 37.0, 1.0], 'GSM3494889': [nan, 61.0, 1.0], 'GSM3494890': [None, None, None], 'GSM3494891': [nan, 31.0, 1.0], 'GSM3494892': [nan, 56.0, 0.0], 'GSM3494893': [nan, 41.0, 0.0], 'GSM3494894': [nan, 61.0, 0.0], 'GSM3494895': [1, None, None], 'GSM3494896': [1.0, 80.0, 1.0], 'GSM3494897': [1.0, 53.0, 1.0], 'GSM3494898': [1.0, 61.0, 1.0], 'GSM3494899': [1.0, 73.0, 1.0], 'GSM3494900': [1.0, 60.0, 1.0], 'GSM3494901': [1.0, 76.0, 1.0], 'GSM3494902': [1.0, 77.0, 0.0], 'GSM3494903': [1.0, 74.0, 0.0], 'GSM3494904': [1.0, 69.0, 1.0], 'GSM3494905': [nan, 77.0, 0.0], 'GSM3494906': [nan, 81.0, 0.0], 'GSM3494907': [nan, 70.0, 0.0], 'GSM3494908': [nan, 82.0, 0.0], 'GSM3494909': [nan, 69.0, 0.0], 'GSM3494910': [nan, 82.0, 0.0], 'GSM3494911': [nan, 67.0, 0.0], 'GSM3494912': [nan, 67.0, 0.0], 'GSM3494913': [nan, 78.0, 0.0], 'GSM3494914': [nan, 67.0, 0.0], 'GSM

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}


### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Identify the keys for mapping
    identifier_key = 'ID'
    gene_symbol_key = 'ENTREZ_GENE_ID'

    # 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function.
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping with the 'apply_gene_mapping' function from the library.
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
